In [1]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from scipy.sparse import random
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator


# Create simulated data

In [2]:
np.random.seed(4)
n = 1000000
features = 500
X = random(n, features, 0.1)


In [3]:
X.shape

(1000000, 500)

In [4]:
y = np.random.choice([0,1], n)
y.reshape(-1,1)
y.shape

(1000000,)

# SKLearn Logistic regression

In [59]:
lr_model = LogisticRegression(penalty='l1', solver='liblinear')
%time lr_model.fit(X, y)

CPU times: user 1min 59s, sys: 2.56 s, total: 2min 2s
Wall time: 2min 4s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [22]:
%time lr_model.score(X,y)

NameError: name 'lr_model' is not defined

# H2O Lasso regression

In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_151"; Java(TM) SE Runtime Environment (build 1.8.0_151-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.151-b12, mixed mode)
  Starting server from /Users/madhan/anaconda/envs/fastaiV2/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/1y/lhg_qvyx5lvb_6x50gczc6mr0000gn/T/tmp1wcnmmox
  JVM stdout: /var/folders/1y/lhg_qvyx5lvb_6x50gczc6mr0000gn/T/tmp1wcnmmox/h2o_madhan_started_from_python.out
  JVM stderr: /var/folders/1y/lhg_qvyx5lvb_6x50gczc6mr0000gn/T/tmp1wcnmmox/h2o_madhan_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.16.0.3
H2O cluster version age:,1 day
H2O cluster name:,H2O_from_python_madhan_3jhyu3
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [12]:
X_h2o = h2o.H2OFrame(X)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [13]:
predictors = X_h2o.columns

In [14]:
y_h2o = h2o.H2OFrame(y, column_names=['y'])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [16]:
h20_frame = X_h2o.cbind(y_h2o)

In [18]:
glm_model = H2OGeneralizedLinearEstimator(family = 'binomial', alpha=1, solver='L_BFGS')


In [19]:
%time glm_model.train(x=predictors, y='y', training_frame=h20_frame)


glm Model Build progress: |███████████████████████████████████████████████| 100%
CPU times: user 88 ms, sys: 25.8 ms, total: 114 ms
Wall time: 5.03 s


In [20]:
%time glm_model.accuracy()

CPU times: user 343 µs, sys: 38 µs, total: 381 µs
Wall time: 397 µs


[[0.4999788574228196, 0.507154]]

# Vopal Wabbit Lasso regression

In [21]:
from vowpalwabbit import pyvw

ModuleNotFoundError: No module named 'vowpalwabbit'